# SpinToolkit

Demo code for Spin wave theory for complex spin orders I: linear spin wave, generalized linear spin wave, multi-magnon continuum and finite-temperature effects_

Authors: Lei Xu, Xiaojian Shi, Yangjie Jiao, and [Zhentao Wang](https://orcid.org/0000-0001-7442-2933)

In this demo, we compute the multi-boson excitation of the S$^2$ skyrmion crystal

## Prerequisites

import the modules and set the number of threads for parallelling

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import SpinToolkit_py as sptk

In [ ]:
sptk.print_system_info()

n_jobs = 6 # set manually
# n_jobs = n_jobs   # use the maximum number of threads available

## Define lattice and hamiltonian

- triangular lattice

- $J_{1}-J_{2}$ Heisenberg model with single-ion anisotropy and Zeeman field

- use "SU(N)" mode

In [ ]:
latt = sptk.lattice(name = "triangular", L = [6, 6])
Nsites = latt.Nsites()

In [ ]:
# Parameters following A. O. Leonov and M. Mostovoy, Nat. Commun. 6, 8275 (2015).
J1 = -1.0  # 1st-neighbor exchange
J2 = 0.5   # 2nd neighbor exchange
K  = 0.2   # easy-axis single-ion anisotropy
h  = 0.2   # Zeeman field

hamiltonian = sptk.model_spin(S = 1.0, mode = "dipole", lattice = latt)
print()

for site_i in range(Nsites):
        coor_i, sub_i = latt.site2coor(site = site_i)
        coor0_i, Mi   = latt.coor2supercell0(coor = coor_i)
        xi = coor_i[0]
        yi = coor_i[1]

        # 1st neighbor terms
        coor_j      = [xi + 1, yi]
        coor0_j, Mj = latt.coor2supercell0(coor = coor_j)
        site_j      = latt.coor2site(coor = coor_j, sub = 0)
        hamiltonian.add_2spin_Jmatrix_XXZ(J = sptk.Vec3(J1, J1, J1),
                                          site_i = site_i, site_j = site_j,
                                          Mi = Mi, Mj = Mj)

        coor_j      = [xi, yi + 1]
        coor0_j, Mj = latt.coor2supercell0(coor = coor_j)
        site_j      = latt.coor2site(coor = coor_j, sub = 0)
        hamiltonian.add_2spin_Jmatrix_XXZ(J = sptk.Vec3(J1, J1, J1),
                                          site_i = site_i, site_j = site_j,
                                          Mi = Mi, Mj = Mj)

        coor_j      = [xi + 1, yi + 1]
        coor0_j, Mj = latt.coor2supercell0(coor = coor_j)
        site_j      = latt.coor2site(coor = coor_j, sub = 0)
        hamiltonian.add_2spin_Jmatrix_XXZ(J = sptk.Vec3(J1, J1, J1),
                                          site_i = site_i, site_j = site_j,
                                          Mi = Mi, Mj = Mj)

        # 2nd neighbor terms
        coor_j      = [xi + 2, yi + 1]
        coor0_j, Mj = latt.coor2supercell0(coor = coor_j)
        site_j      = latt.coor2site(coor = coor_j, sub = 0)
        hamiltonian.add_2spin_Jmatrix_XXZ(J = sptk.Vec3(J2, J2, J2),
                                          site_i = site_i, site_j = site_j,
                                          Mi = Mi, Mj = Mj)

        coor_j      = [xi + 1, yi + 2]
        coor0_j, Mj = latt.coor2supercell0(coor = coor_j)
        site_j      = latt.coor2site(coor = coor_j, sub = 0)
        hamiltonian.add_2spin_Jmatrix_XXZ(J = sptk.Vec3(J2, J2, J2),
                                          site_i = site_i, site_j = site_j,
                                          Mi = Mi, Mj = Mj)

        coor_j      = [xi - 1, yi + 1]
        coor0_j, Mj = latt.coor2supercell0(coor = coor_j)
        site_j      = latt.coor2site(coor = coor_j, sub = 0)
        hamiltonian.add_2spin_Jmatrix_XXZ(J = sptk.Vec3(J2, J2, J2),
                                          site_i = site_i, site_j = site_j,
                                          Mi = Mi, Mj = Mj)

        # single-ion anisotropy
        hamiltonian.add_site_Amatrix(D = sptk.Vec3(0.0, 0.0, -0.5 * K), site = site_i)

        # Zeeman field
        hamiltonian.add_zeeman(h = sptk.Vec3(0.0, 0.0, h), site = site_i)

hamiltonian.simplify()
print()
hamiltonian.build_mc_list()
print()

## Perform minimization (minimize from several independent initial random guesses, then pick the lowest-energy solution)

In [ ]:
print("Optimizing ground state", end = "... ", flush = True)
start        = time.perf_counter()
s_min, f_min = hamiltonian.optimize_spins_dipole(total_seeds = 200) # more seeds -> more likely to hit global minimum
end          = time.perf_counter()
print(f"{end - start}s")
print()

print(f"f_min (global): {f_min}")
for i in range(Nsites):
    print(f"site-{i}: {s_min[i]}")

## Initialize the LSW calculation and prepare the rotation matrices $\vec{s_i} = R_i \hat z$

In [ ]:
R = hamiltonian.init_LSW(s = s_min)
for i in range(3):
    print(f"site-{i}:")
    print(R[i])
    print()

## Define a $k$-space trajectory, a list of $\omega$ and Gaussian broadening factor $\sigma$ for computing $\mathcal{S}^{ab}({\bm k}, \omega)$

In [ ]:
kc = sptk.k_cut(lattice = latt, density = 200)
kc.add_k(k_new = [0.0, 0.0])               # Gamma
kc.add_k(k_new = [0.5, 0.0])               # M
xtic_M = kc.len_list[-1]                   # x-axis position for M-point (for plotting)
kc.add_k(k_new = [1.0 / 3.0, 1.0 / 3.0])   # K
xtic_K = kc.len_list[-1]                   # x-axis position for K-point (for plotting)
kc.add_k(k_new = [0.0, 0.0])               # Gamma

ω_max = 12.0
num_ω = 600
ω_list = np.arange(0, ω_max, ω_max / num_ω, dtype = np.float64).tolist()

FWHM  = 0.05
sigma = FWHM / 2.35482

## Compute 1-magnon DSSF $\mathcal{S}_{1}^{ab}({\bm k},\omega)$

In [ ]:
omega_k  = []
Sxx      = []
Syy      = []
Szz      = []
SxyPyx_R = []
SyzPzy_R = []
SzxPxz_R = []
SxyMyx_I = []
SyzMzy_I = []
SzxMxz_I = []

start = time.perf_counter()
total_k = kc.size()
for index_k in range(total_k):
    k = kc.k_list[index_k]
    if index_k % sptk.round2int(total_k / 50.0 + 1.0) == 0:
        print("*", end = "", flush = True)
    try:
        disp0, Sxx0, Syy0, Szz0, \
        SxyPyx_R0, SyzPzy_R0, SzxPxz_R0, \
        SxyMyx_I0, SyzMzy_I0, SzxMxz_I0 \
        = sptk.DSSF_LSW(model = hamiltonian, R = R, T = 0.0,
                        k = k, omega_list = ω_list,
                        eval_1magnon = True, maxeval_2magnon = 0,
                        maxeval_3magnon = 0, maxeval_0magnon = 0,
                        broadening = "Gaussian", sigma_or_eta = sigma,
                        epsilon = 1.0e-4)
        omega_k.append(disp0)
        Sxx.append(Sxx0)
        Syy.append(Syy0)
        Szz.append(Szz0)
        SxyPyx_R.append(SxyPyx_R0)
        SyzPzy_R.append(SyzPzy_R0)
        SzxPxz_R.append(SzxPxz_R0)
        SxyMyx_I.append(SxyMyx_I0)
        SyzMzy_I.append(SyzMzy_I0)
        SzxMxz_I.append(SzxMxz_I0)
    except Exception as e:
        print(f"error: {e}, k = {k}")
print()
end = time.perf_counter()
print(f"Used {end - start:.6f}s")

## Plot 1-magnon DSSF $\mathcal{S}_{1}({\bm k}, \omega) \equiv \mathcal{S}_{1}^{xx}({\bm k}, \omega) + \mathcal{S}_{1}^{yy}({\bm k}, \omega) + \mathcal{S}_{1}^{zz}({\bm k}, \omega)$

In [ ]:
plt.rcParams['figure.figsize'] = (6, 4)
plt.rcParams['figure.facecolor'] = 'none'
plt.rcParams['font.size'] = 15

plt.rcParams['text.usetex'] = True
plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath,newtxtext,newtxmath,bm}'
plt.rcParams['font.family'] = 'TeX Gyre Termes'

x = kc.len_list
axis_X, axis_Y = np.meshgrid(x, ω_list)
Intensity = (np.array(Sxx) + np.array(Syy) + np.array(Szz)).transpose()

fig, ax = plt.subplots()

cmap = colors.LinearSegmentedColormap.from_list("white_to_blue", [(1, 1, 1), (0, 0, 1)], N = 256)
imag = ax.pcolormesh(axis_X, axis_Y, Intensity, cmap = cmap, shading = "auto", vmin = 0, vmax = 30)

ax.set_xlim(x[0], x[-1])
ax.set_ylim(0.0, 12.0)
ax.set_xticks([x[0], xtic_M, xtic_K, x[-1]], [r"$(0,0)$", r"$(\frac{1}{2},0)$", r"$(\frac{1}{3},\frac{1}{3})$", r"$(0,0)$"])
ax.set_yticks([0, 4, 8, 12])
ax.set_xlabel(r"$\bm{k}$")
ax.set_ylabel(r"$\omega/J$")
ax.tick_params(direction = 'in', color = 'black')

ax.text(6.7, 11.1, r'$\mathcal{S}_1(\bm{k},\omega)$', fontsize = 15)
ax.text(5.6, 10.3, r'$0$', fontsize = 15)
ax.text(9.2, 10.3, r'$30$', fontsize = 15)

cax = ax.inset_axes([6.0, 10.3, 3.0, 0.5], transform = ax.transData)
cbar = fig.colorbar(imag, cax = cax, orientation = "horizontal")
cbar.set_ticks([0, 30])
cbar.set_ticklabels(['', ''])
cbar.ax.tick_params(length = 0)

#plt.savefig("Fig_SkX_S2.pdf", bbox_inches = 'tight', pad_inches = 0)
plt.show()